In [169]:
import numpy as np
import pandas as pd

__Importing the Car Evaluation Dataset__

In [171]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
y = car_evaluation.data.targets 

In [172]:
print(type(X))
print(type(y))

print(X.shape)
print(y.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(1728, 6)
(1728, 1)


__One Hot Encoding__

In [174]:
X_encoded = pd.get_dummies(X) # One hot encoding
y_encoded = pd.get_dummies(y) # One hot encoding

# print(X_encoded)
# print(y_encoded)

print(type(X))
print(type(y))

print(X_encoded.shape)
print(y_encoded.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(1728, 21)
(1728, 4)


__Dataset Partitioning__

In [176]:
from sklearn.model_selection import train_test_split

# data split, 70% training and 30% temp (temp = validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

# 30% temp data into 15% validation and 15% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

__Building a Single Decision Tree__

In [178]:
from sklearn.metrics import confusion_matrix
import numpy as np

def calculate_metrics(y_true, y_pred):
    # Initialize confusion matrix counts
    acc_acc = acc_good = acc_unacc = acc_vgood = 0
    good_acc = good_good = good_unacc = good_vgood = 0
    unacc_acc = unacc_good = unacc_unacc = unacc_vgood = 0
    vgood_acc = vgood_good = vgood_unacc = vgood_vgood = 0

    y_true = y_true.to_numpy()
    
    # # Calculate confusion matrix manually
    # for actual, predicted in zip(y_true, y_pred):
    for i in range(len(y_true)):
        actual = y_true[i]
        # actual = y_true.iloc[i].values # <class 'pandas.core.frame.DataFrame'>
        predicted = y_pred[i] # <class 'numpy.ndarray'>
        
        if actual[0] and predicted[0]:
            acc_acc += 1
        elif actual[0] and predicted[1]:
            acc_good += 1
        elif actual[0] and predicted[2]:
            acc_unacc += 1
        elif actual[0] and predicted[3]:
            acc_vgood += 1
        elif actual[1] and predicted[0]:
            good_acc += 1
        elif actual[1] and predicted[1]:
            good_good += 1
        elif actual[1] and predicted[2]:
            good_unacc += 1
        elif actual[1] and predicted[3]:
            good_vgood += 1
        elif actual[2] and predicted[0]:
            unacc_acc += 1
        elif actual[2] and predicted[1]:
            unacc_good += 1
        elif actual[2] and predicted[2]:
            unacc_unacc += 1
        elif actual[2] and predicted[3]:
            unacc_vgood += 1
        elif actual[3] and predicted[0]:
            vgood_acc += 1
        elif actual[3] and predicted[1]:
            vgood_good += 1
        elif actual[3] and predicted[2]:
            vgood_unacc += 1
        elif actual[3] and predicted[3]:
            vgood_vgood += 1

    # Confusion matrix as an array
    confusion_matrix = [
        [acc_acc, acc_good, acc_unacc, acc_vgood],
        [good_acc, good_good, good_unacc, good_vgood],
        [unacc_acc, unacc_good, unacc_unacc, unacc_vgood],
        [vgood_acc, vgood_good, vgood_unacc, vgood_vgood]
    ]

    # Accuracy
    total_correct = acc_acc + good_good + unacc_unacc + vgood_vgood
    total_predictions = sum(sum(row) for row in confusion_matrix)
    accuracy = total_correct / total_predictions

    # Precision calculations
    Precision_of_acc = acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc)
    Precision_of_good = good_good / (acc_good + good_good + unacc_good + vgood_good)
    Precision_of_unacc = unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc)
    Precision_of_vgood = vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood)
    average_precision = (Precision_of_acc + Precision_of_good + Precision_of_unacc + Precision_of_vgood) / 4.0

    # Recall calculations
    Recall_of_acc = acc_acc / (acc_acc + acc_good + acc_unacc + acc_vgood)
    Recall_of_good = good_good / (good_acc + good_good + good_unacc + good_vgood)
    Recall_of_unacc = unacc_unacc / (unacc_acc + unacc_good + unacc_unacc + unacc_vgood)
    Recall_of_vgood = vgood_vgood / (vgood_acc + vgood_good + vgood_unacc + vgood_vgood)
    average_recall = (Recall_of_acc + Recall_of_good + Recall_of_unacc + Recall_of_vgood) / 4.0

    # F1 Score
    average_f1_score = 2 * ((average_precision * average_recall) / (average_precision + average_recall))

    # Print or return metrics
    print("Confusion Matrix:")
    for row in confusion_matrix:
        print(row)

    print(f"Accuracy: {accuracy}")
    print(f"Average Precision: {average_precision}")
    print(f"Average Recall: {average_recall}")
    print(f"Average F1 Score: {average_f1_score}")

    # return {
    #     'accuracy': accuracy,
    #     'average_precision': average_precision,
    #     'average_recall': average_recall,
    #     'average_f1_score': average_f1_score,
    #     'confusion_matrix': confusion_matrix
    # }

model1 = DecisionTreeClassifier()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_val)
print("\nMetrics for Model 1:")
calculate_metrics(y_val, y_pred1)




Metrics for Model 1:
Confusion Matrix:
[50, 2, 3, 2]
[0, 9, 0, 1]
[0, 0, 178, 0]
[1, 1, 0, 12]
Accuracy: 0.9613899613899614
Average Precision: 0.8784543928068465
Average Recall: 0.9085839598997494
Average F1 Score: 0.8932651832557713


In [179]:
from sklearn.metrics import confusion_matrix
import numpy as np


# for i in range(rows):
#     actual = y_val.iloc[i].values # <class 'pandas.core.frame.DataFrame'>
#     predicted = y_val_pred[i] # <class 'numpy.ndarray'>

def calculate_metrics(y_true, y_pred):
    # Initialize confusion matrix counts
    acc_acc = acc_good = acc_unacc = acc_vgood = 0
    good_acc = good_good = good_unacc = good_vgood = 0
    unacc_acc = unacc_good = unacc_unacc = unacc_vgood = 0
    vgood_acc = vgood_good = vgood_unacc = vgood_vgood = 0

    # Calculate confusion matrix manually
    for i in range(rows):
        actual = y_true.iloc[i].values # <class 'pandas.core.frame.DataFrame'>
        predicted = y_pred[i] # <class 'numpy.ndarray'>
    # for actual, predicted in zip(y_true, y_pred):
        if actual == 0 and predicted == 0:
            acc_acc += 1
        elif actual == 0 and predicted == 1:
            acc_good += 1
        elif actual == 0 and predicted == 2:
            acc_unacc += 1
        elif actual == 0 and predicted == 3:
            acc_vgood += 1
        elif actual == 1 and predicted == 0:
            good_acc += 1
        elif actual == 1 and predicted == 1:
            good_good += 1
        elif actual == 1 and predicted == 2:
            good_unacc += 1
        elif actual == 1 and predicted == 3:
            good_vgood += 1
        elif actual == 2 and predicted == 0:
            unacc_acc += 1
        elif actual == 2 and predicted == 1:
            unacc_good += 1
        elif actual == 2 and predicted == 2:
            unacc_unacc += 1
        elif actual == 2 and predicted == 3:
            unacc_vgood += 1
        elif actual == 3 and predicted == 0:
            vgood_acc += 1
        elif actual == 3 and predicted == 1:
            vgood_good += 1
        elif actual == 3 and predicted == 2:
            vgood_unacc += 1
        elif actual == 3 and predicted == 3:
            vgood_vgood += 1

    # Confusion matrix as an array
    confusion_matrix = [
        [acc_acc, acc_good, acc_unacc, acc_vgood],
        [good_acc, good_good, good_unacc, good_vgood],
        [unacc_acc, unacc_good, unacc_unacc, unacc_vgood],
        [vgood_acc, vgood_good, vgood_unacc, vgood_vgood]
    ]

    # Accuracy
    total_correct = acc_acc + good_good + unacc_unacc + vgood_vgood
    total_predictions = sum(sum(row) for row in confusion_matrix)
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0

    # Precision calculations (with manual check for division by zero)
    Precision_of_acc = acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc) if (acc_acc + good_acc + unacc_acc + vgood_acc) != 0 else 0
    Precision_of_good = good_good / (acc_good + good_good + unacc_good + vgood_good) if (acc_good + good_good + unacc_good + vgood_good) != 0 else 0
    Precision_of_unacc = unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc) if (acc_unacc + good_unacc + unacc_unacc + vgood_unacc) != 0 else 0
    Precision_of_vgood = vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood) if (acc_vgood + good_vgood + unacc_vgood + vgood_vgood) != 0 else 0

    # Average precision
    average_precision = (Precision_of_acc + Precision_of_good + Precision_of_unacc + Precision_of_vgood) / 4.0

    # Recall calculations (with manual check for division by zero)
    Recall_of_acc = acc_acc / (acc_acc + acc_good + acc_unacc + acc_vgood) if (acc_acc + acc_good + acc_unacc + acc_vgood) != 0 else 0
    Recall_of_good = good_good / (good_acc + good_good + good_unacc + good_vgood) if (good_acc + good_good + good_unacc + good_vgood) != 0 else 0
    Recall_of_unacc = unacc_unacc / (unacc_acc + unacc_good + unacc_unacc + unacc_vgood) if (unacc_acc + unacc_good + unacc_unacc + unacc_vgood) != 0 else 0
    Recall_of_vgood = vgood_vgood / (vgood_acc + vgood_good + vgood_unacc + vgood_vgood) if (vgood_acc + vgood_good + vgood_unacc + vgood_vgood) != 0 else 0

    # Average recall
    average_recall = (Recall_of_acc + Recall_of_good + Recall_of_unacc + Recall_of_vgood) / 4.0

    # F1 Score calculation (with manual check for division by zero)
    average_f1_score = 2 * ((average_precision * average_recall) / (average_precision + average_recall)) if (average_precision + average_recall) != 0 else 0

    # Print or return metrics
    print("Confusion Matrix:")
    for row in confusion_matrix:
        print(row)

    print(f"Accuracy: {accuracy}")
    print(f"Average Precision: {average_precision}")
    print(f"Average Recall: {average_recall}")
    print(f"Average F1 Score: {average_f1_score}")

    # return {
    #     'accuracy': accuracy,
    #     'average_precision': average_precision,
    #     'average_recall': average_recall,
    #     'average_f1_score': average_f1_score,
    #     'confusion_matrix': confusion_matrix
    # }



model1 = DecisionTreeClassifier()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_val)
print("\nMetrics for Model 1:")
calculate_metrics(y_val, y_pred1)


Metrics for Model 1:


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# # Define models
# model1 = DecisionTreeClassifier(max_depth=10, min_samples_split=4, criterion='gini', min_samples_leaf=1, random_state=42)
# model2 = DecisionTreeClassifier(max_depth=9, min_samples_split=5, criterion='entropy', min_samples_leaf=4, random_state=42)
# model3 = DecisionTreeClassifier(max_depth=5, min_samples_split=2, criterion='gini', min_samples_leaf=10, random_state=42)
# model4 = DecisionTreeClassifier(max_depth=11, min_samples_split=2, criterion='entropy', min_samples_leaf=1, random_state=42)
model1 = DecisionTreeClassifier()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_val)
print("\nMetrics for Model 1:")
calculate_metrics(y_val, y_pred1)

# # Train and evaluate model 1
# model1.fit(X_train, y_train)
# y_pred1 = model1.predict(X_val)
# print("\nMetrics for Model 1:")
# calculate_metrics(y_val, y_pred1)

# # Train and evaluate model 2
# model2.fit(X_train, y_train)
# y_pred2 = model2.predict(X_val)
# print("\nMetrics for Model 2:")
# calculate_metrics(y_val, y_pred2)

# # Train and evaluate model 3
# model3.fit(X_train, y_train)
# y_pred3 = model3.predict(X_val)
# print("\nMetrics for Model 3:")
# calculate_metrics(y_val, y_pred3)

# # Train and evaluate model 4
# model4.fit(X_train, y_train)
# y_pred4 = model4.predict(X_val)
# print("\nMetrics for Model 4:")
# calculate_metrics(y_val, y_pred4)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
single_decision_tree_classifier = DecisionTreeClassifier()

In [ ]:
single_decision_tree_classifier.fit(X_train, y_train)

In [ ]:
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt

# plt.figure(figsize=(40, 20))
# plot_tree(single_decision_tree_classifier, filled=True, feature_names=X_train.columns, class_names=single_decision_tree_classifier.classes_)
# plt.show()

In [ ]:
# from sklearn.model_selection import GridSearchCV

# dt_params = {'max_depth': [3,4,5,6,7,8,9,10,11], 'min_samples_split': [2,3,4,5,6,7,8,9,10,11],'min_samples_leaf':[1,2,3,4,5],'random_state':[42]}
# dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5, scoring='accuracy')
# dt_grid.fit(X_train, y_train)
# best_params = dt_grid.best_params_
# print(f"Best Hyperparameters: {best_params}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model1 = DecisionTreeClassifier(max_depth=10, min_samples_split=4, criterion='gini', min_samples_leaf=1, random_state=42)
model2 = DecisionTreeClassifier(max_depth=9, min_samples_split=5, criterion='entropy', min_samples_leaf=4, random_state=42)
model3 = DecisionTreeClassifier(max_depth=5, min_samples_split=2, criterion='gini', min_samples_leaf=10, random_state=42)
model4 = DecisionTreeClassifier(max_depth=11, min_samples_split=2, criterion='entropy', min_samples_leaf=1, random_state=42)


# Fit models
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)


# Predictions on validation set
y_pred_val1 = model1.predict(X_val)
y_pred_val2 = model2.predict(X_val)
y_pred_val3 = model3.predict(X_val)
y_pred_val4 = model4.predict(X_val)


# Validation results for model1
val_accuracy = accuracy_score(y_val, y_pred_val1)
val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
    y_val, y_pred_val1, average='macro', zero_division=1
)
print("-------------------------------------------")
print("Validation results for Model 1: ")
print(f"Accuracy: {val_accuracy}")
print(f"Average Precision: {val_precision}")
print(f"Average Recall: {val_recall}")
print(f"Average F1-score: {val_f1}")


# Validation results for model2
val_accuracy = accuracy_score(y_val, y_pred_val2)
val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
    y_val, y_pred_val2, average='macro', zero_division=1
)
print("-------------------------------------------")
print("Validation results for Model 2: ")
print(f"Accuracy: {val_accuracy}")
print(f"Average Precision: {val_precision}")
print(f"Average Recall: {val_recall}")
print(f"Average F1-score: {val_f1}")


# Validation results for model3
val_accuracy = accuracy_score(y_val, y_pred_val3)
val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
    y_val, y_pred_val3, average='macro', zero_division=1
)
print("-------------------------------------------")
print("Validation results for Model 3: ")
print(f"Accuracy: {val_accuracy}")
print(f"Average Precision: {val_precision}")
print(f"Average Recall: {val_recall}")
print(f"Average F1-score: {val_f1}")


# Validation results for model4
val_accuracy = accuracy_score(y_val, y_pred_val4)
val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
    y_val, y_pred_val4, average='macro', zero_division=1
)
print("-------------------------------------------")
print("Validation results for Model 4: ")
print(f"Accuracy: {val_accuracy}")
print(f"Average Precision: {val_precision}")
print(f"Average Recall: {val_recall}")
print(f"Average F1-score: {val_f1}")


# Test set results for model4
y_pred_test = model4.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
test_precision, test_recall, test_f1, test_support = precision_recall_fscore_support(
    y_test, y_pred_test, average='macro', zero_division=1
)
print("-------------------------------------------")
print("Test Sets Results using Model 4: ")
print(f"Accuracy: {test_accuracy}")
print(f"Average Precision: {test_precision}")
print(f"Average Recall: {test_recall}")
print(f"Average F1-score: {test_f1}")

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV

# # Hyperparameter Tuning for Decision Tree Classifier
# dt_params = {'max_depth': [10, 11, 12, 13, 14, 15, 16], 'min_samples_split': [2, 3, 4, 5, 5, 6, 7, 8, 9]}
# dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=5, scoring='accuracy')
# dt_grid.fit(X_train, y_train)

# best_params = dt_grid.best_params_
# print(f"Best Hyperparameters: {best_params}")

In [ ]:
# # Step 1: Retrieve the best parameters from GridSearchCV
# # best_params = dt_grid.best_params_

# # Step 2: Initialize a new Decision Tree Classifier with the best parameters
# best_dt_model = DecisionTreeClassifier(**best_params)

# # Step 3: Fit the model on the training set
# best_dt_model.fit(X_train, y_train)

# # Step 4: Evaluate on the validation set
# y_val_pred = best_dt_model.predict(X_val)
# val_accuracy = accuracy_score(y_val, y_val_pred)
# print(f"Validation Accuracy: {val_accuracy}")

# # Step 5: Evaluate on the test set
# y_test_pred = best_dt_model.predict(X_test)
# test_accuracy = accuracy_score(y_test, y_test_pred)
# print(f"Test Accuracy: {test_accuracy}")

__Validation Set__

In [ ]:
# y_val contains actual ground truth labels of the validation set
# y_val_pred is our model's prediction on the X_val (validation set)

y_val_pred = single_decision_tree_classifier.predict(X_val)

print(y_val)
print(type(y_val))
print(y_val.shape)

print(y_val_pred)
print(type(y_val_pred))
print(y_val_pred.shape)

In [ ]:
y_val_rows, y_val_cols = y_val.shape
print(y_val_rows)
print(y_val_cols)

y_val_pred_rows, y_val_pred_cols = y_val_pred.shape
print(y_val_pred_rows)
print(y_val_pred_cols)

if y_val_rows == y_val_pred_rows:
    rows = y_val_rows
    print(f"rows = {rows}")

if y_val_cols == y_val_pred_cols:
    cols = y_val_cols
    print(f"columns = {cols}")

__Manually building the Confusion Matrix on the Validation Set__

In [ ]:
acc_acc = acc_good = acc_unacc = acc_vgood = 0
good_acc = good_good = good_unacc = good_vgood = 0
unacc_acc = unacc_good = unacc_unacc = unacc_vgood = 0
vgood_acc = vgood_good = vgood_unacc = vgood_vgood = 0

for i in range(rows):
    actual = y_val.iloc[i].values # <class 'pandas.core.frame.DataFrame'>
    predicted = y_val_pred[i] # <class 'numpy.ndarray'>
    
    if actual[0] == True and predicted[0] == True:
        acc_acc += 1
    elif actual[0] == True and predicted[1] == True:
        acc_good += 1
    elif actual[0] == True and predicted[2] == True:
        acc_unacc += 1
    elif actual[0] == True and predicted[3] == True:
        acc_vgood += 1
    
    if actual[1] == True and predicted[0] == True:
        good_acc += 1
    elif actual[1] == True and predicted[1] == True:
        good_good += 1
    elif actual[1] == True and predicted[2] == True:
        good_unacc += 1
    elif actual[1] == True and predicted[3] == True:
        good_vgood += 1

    if actual[2] == True and predicted[0] == True:
        unacc_acc += 1
    elif actual[2] == True and predicted[1] == True:
        unacc_good += 1
    elif actual[2] == True and predicted[2] == True:
        unacc_unacc += 1
    elif actual[2] == True and predicted[3] == True:
        unacc_vgood += 1

    if actual[3] == True and predicted[0] == True:
        vgood_acc += 1
    elif actual[3] == True and predicted[1] == True:
        vgood_good += 1
    elif actual[3] == True and predicted[2] == True:
        vgood_unacc += 1
    elif actual[3] == True and predicted[3] == True:
        vgood_vgood += 1

print("Confusion Matrix:")
print(f"[{acc_acc}, {acc_good}, {acc_unacc}, {acc_vgood}]")
print(f"[{good_acc}, {good_good}, {good_unacc}, {good_vgood}]")
print(f"[{unacc_acc}, {unacc_good}, {unacc_unacc}, {unacc_vgood}]")
print(f"[{vgood_acc}, {vgood_good}, {vgood_unacc}, {vgood_vgood}]")

__Creating a List Containing the Confusion Matrix of Validation Set__

In [ ]:
confusion_matrix = [[acc_acc, acc_good, acc_unacc, acc_vgood],
                   [good_acc, good_good, good_unacc, good_vgood],
                   [unacc_acc, unacc_good, unacc_unacc, unacc_vgood],
                   [vgood_acc, vgood_good, vgood_unacc, vgood_vgood]]
print(confusion_matrix)
print(type(confusion_matrix))

__Manually Calculating the Validation Set's Accuracy from the Manually Built Confusion Matrix__

In [ ]:
print((acc_acc + good_good + unacc_unacc + vgood_vgood) / sum(sum(row) for row in confusion_matrix))

# print((acc_acc + good_good + unacc_unacc + vgood_vgood) / (acc_acc + acc_good + acc_unacc + acc_vgood + good_acc + good_good + good_unacc + good_vgood + unacc_acc + unacc_good + unacc_unacc + unacc_vgood + vgood_acc + vgood_good + vgood_unacc + vgood_vgood))

__Accuracy of the Single Decision Tree on the Validation Set__ `using the Sklearn Implementation`

In [ ]:
from sklearn.metrics import accuracy_score

# y_val_pred = single_decision_tree_classifier.predict(X_val)

val_accuracy = accuracy_score(y_val, y_val_pred)
print(val_accuracy)

__Manually Calculating the Validation Set's Precision & Recall from the Manually Built Confusion Matrix__

In [ ]:
# Precision of acc
# print(acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc))
Precision_of_acc = acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc)
print(f"Precision of acc: {Precision_of_acc}")

# Precision of good
# print(good_good / (acc_good + good_good + unacc_good + vgood_good))
Precision_of_good = good_good / (acc_good + good_good + unacc_good + vgood_good)
print(f"Precision of good: {Precision_of_good}")

# Precision of unacc
# print(unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc))
Precision_of_unacc = unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc)
print(f"Precision of unacc: {Precision_of_unacc}")

# Precision of vgood
# print(vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood))
Precision_of_vgood = vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood)
print(f"Precision of vgood: {Precision_of_vgood}")

# Average precision
average_precision = (Precision_of_acc + Precision_of_good + Precision_of_unacc + Precision_of_vgood) / 4.0
print("---------------------------------------")
print(f"Average Precision: {average_precision}")
print("---------------------------------------")

# Recall of acc
Recall_of_acc = acc_acc / (acc_acc + acc_good + acc_unacc + acc_vgood)
print(f"Recall of acc: {Recall_of_acc}")

# Recall of good
Recall_of_good = good_good / (good_acc + good_good + good_unacc + good_vgood)
print(f"Recall of good: {Recall_of_good}")

# Recall of unacc
Recall_of_unacc = unacc_unacc / (unacc_acc + unacc_good + unacc_unacc + unacc_vgood)
print(f"Recall of unacc: {Recall_of_unacc}")

# Recall of vgood
Recall_of_vgood = vgood_vgood / (vgood_acc + vgood_good + vgood_unacc + vgood_vgood)
print(f"Recall of vgood: {Recall_of_vgood}")

# Average recall
average_recall = (Recall_of_acc + Recall_of_good + Recall_of_unacc + Recall_of_vgood) / 4.0
print("---------------------------------------")
print(f"Average recall: {average_recall}")
print("---------------------------------------")

# Average F1 Score
average_f1_score = 2 * ((average_precision * average_recall) / (average_precision + average_recall))
print("---------------------------------------")
print(f"Average F1 Score: {average_f1_score}")
print("---------------------------------------")

__Confusion Matrix, Precision, Recall, F1 Score on the Validation Set__ `using the Sklearn Implementation`

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_val_labels = np.argmax(y_val.values, axis=1)
y_val_pred_labels = np.argmax(y_val_pred, axis=1)

conf_matrix = confusion_matrix(y_val_labels, y_val_pred_labels)

print("Confusion Matrix using sklearn:")
print(conf_matrix)

print("Classification Report:")
print(classification_report(y_val_labels, y_val_pred_labels, zero_division=0))

__Test Set__

In [ ]:
# y_test contains actual ground truth labels of the test set
# y_test_pred is our model's prediction on the X_test (Test set)

y_test_pred = single_decision_tree_classifier.predict(X_test)

print(y_test)
print(type(y_test))
print(y_test.shape)

print(y_test_pred)
print(type(y_test_pred))
print(y_test_pred.shape)

In [ ]:
y_test_rows, y_test_cols = y_test.shape
print(y_test_rows)
print(y_test_cols)

y_test_pred_rows, y_test_pred_cols = y_test_pred.shape
print(y_test_pred_rows)
print(y_test_pred_cols)

if y_test_rows == y_test_pred_rows:
    rows = y_test_rows
    print(f"rows = {rows}")

if y_test_cols == y_test_pred_cols:
    cols = y_test_cols
    print(f"columns = {cols}")

__Manually building the Confusion Matrix on the Test Set__

In [ ]:
acc_acc = acc_good = acc_unacc = acc_vgood = 0
good_acc = good_good = good_unacc = good_vgood = 0
unacc_acc = unacc_good = unacc_unacc = unacc_vgood = 0
vgood_acc = vgood_good = vgood_unacc = vgood_vgood = 0

for i in range(rows):
    actual = y_test.iloc[i].values # <class 'pandas.core.frame.DataFrame'>
    predicted = y_test_pred[i] # <class 'numpy.ndarray'>
    
    if actual[0] == True and predicted[0] == True:
        acc_acc += 1
    elif actual[0] == True and predicted[1] == True:
        acc_good += 1
    elif actual[0] == True and predicted[2] == True:
        acc_unacc += 1
    elif actual[0] == True and predicted[3] == True:
        acc_vgood += 1
    
    if actual[1] == True and predicted[0] == True:
        good_acc += 1
    elif actual[1] == True and predicted[1] == True:
        good_good += 1
    elif actual[1] == True and predicted[2] == True:
        good_unacc += 1
    elif actual[1] == True and predicted[3] == True:
        good_vgood += 1

    if actual[2] == True and predicted[0] == True:
        unacc_acc += 1
    elif actual[2] == True and predicted[1] == True:
        unacc_good += 1
    elif actual[2] == True and predicted[2] == True:
        unacc_unacc += 1
    elif actual[2] == True and predicted[3] == True:
        unacc_vgood += 1

    if actual[3] == True and predicted[0] == True:
        vgood_acc += 1
    elif actual[3] == True and predicted[1] == True:
        vgood_good += 1
    elif actual[3] == True and predicted[2] == True:
        vgood_unacc += 1
    elif actual[3] == True and predicted[3] == True:
        vgood_vgood += 1

print("Confusion Matrix:")
print(f"[{acc_acc}, {acc_good}, {acc_unacc}, {acc_vgood}]")
print(f"[{good_acc}, {good_good}, {good_unacc}, {good_vgood}]")
print(f"[{unacc_acc}, {unacc_good}, {unacc_unacc}, {unacc_vgood}]")
print(f"[{vgood_acc}, {vgood_good}, {vgood_unacc}, {vgood_vgood}]")

__Creating a List Containing the Confusion Matrix of Test Set__

In [ ]:
confusion_matrix = [[acc_acc, acc_good, acc_unacc, acc_vgood],
                   [good_acc, good_good, good_unacc, good_vgood],
                   [unacc_acc, unacc_good, unacc_unacc, unacc_vgood],
                   [vgood_acc, vgood_good, vgood_unacc, vgood_vgood]]
print(confusion_matrix)
print(type(confusion_matrix))

__Manually Calculating the Test Set's Accuracy from the Manually Built Confusion Matrix__

In [ ]:
print((acc_acc + good_good + unacc_unacc + vgood_vgood) / sum(sum(row) for row in confusion_matrix))

# print((acc_acc + good_good + unacc_unacc + vgood_vgood) / (acc_acc + acc_good + acc_unacc + acc_vgood + good_acc + good_good + good_unacc + good_vgood + unacc_acc + unacc_good + unacc_unacc + unacc_vgood + vgood_acc + vgood_good + vgood_unacc + vgood_vgood))

__Accuracy of the Single Decision Tree on the Test Set__ `using the Sklearn Implementation`

In [ ]:
from sklearn.metrics import accuracy_score

# y_test_pred = single_decision_tree_classifier.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
print(test_accuracy)

__Manually Calculating the Test Set's Precision & Recall from the Manually Built Confusion Matrix__

In [ ]:
# Precision of acc
# print(acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc))
Precision_of_acc = acc_acc / (acc_acc + good_acc + unacc_acc + vgood_acc)
print(f"Precision of acc: {Precision_of_acc}")

# Precision of good
# print(good_good / (acc_good + good_good + unacc_good + vgood_good))
Precision_of_good = good_good / (acc_good + good_good + unacc_good + vgood_good)
print(f"Precision of good: {Precision_of_good}")

# Precision of unacc
# print(unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc))
Precision_of_unacc = unacc_unacc / (acc_unacc + good_unacc + unacc_unacc + vgood_unacc)
print(f"Precision of unacc: {Precision_of_unacc}")

# Precision of vgood
# print(vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood))
Precision_of_vgood = vgood_vgood / (acc_vgood + good_vgood + unacc_vgood + vgood_vgood)
print(f"Precision of vgood: {Precision_of_vgood}")

# Average precision
average_precision = (Precision_of_acc + Precision_of_good + Precision_of_unacc + Precision_of_vgood) / 4.0
print("---------------------------------------")
print(f"Average Precision: {average_precision}")
print("---------------------------------------")

# Recall of acc
Recall_of_acc = acc_acc / (acc_acc + acc_good + acc_unacc + acc_vgood)
print(f"Recall of acc: {Recall_of_acc}")

# Recall of good
Recall_of_good = good_good / (good_acc + good_good + good_unacc + good_vgood)
print(f"Recall of good: {Recall_of_good}")

# Recall of unacc
Recall_of_unacc = unacc_unacc / (unacc_acc + unacc_good + unacc_unacc + unacc_vgood)
print(f"Recall of unacc: {Recall_of_unacc}")

# Recall of vgood
Recall_of_vgood = vgood_vgood / (vgood_acc + vgood_good + vgood_unacc + vgood_vgood)
print(f"Recall of vgood: {Recall_of_vgood}")

# Average recall
average_recall = (Recall_of_acc + Recall_of_good + Recall_of_unacc + Recall_of_vgood) / 4.0
print("---------------------------------------")
print(f"Average recall: {average_recall}")
print("---------------------------------------")

# Average F1 Score
average_f1_score = 2 * ((average_precision * average_recall) / (average_precision + average_recall))
print("---------------------------------------")
print(f"Average F1 Score: {average_f1_score}")
print("---------------------------------------")

__Confusion Matrix, Precision, Recall, F1 Score on the Test Set__ `using the Sklearn Implementation`

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_test_labels = np.argmax(y_test.values, axis=1)
y_test_pred_labels = np.argmax(y_test_pred, axis=1)

conf_matrix = confusion_matrix(y_test_labels, y_test_pred_labels)

print("Confusion Matrix using sklearn:")
print(conf_matrix)

print("Classification Report:")
print(classification_report(y_test_labels, y_test_pred_labels, zero_division=0))

__XGBOOST__

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Create an XGBoost Classifier
xgb_model = XGBClassifier(random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on the validation set
y_pred_val = xgb_model.predict(X_val)

# Evaluate the model on the validation set
precision, recall, f1, support = precision_recall_fscore_support(y_val, y_pred_val, average=None)
print("Validation Precision:", precision)
print("Validation Recall:", recall)
print("Validation F1-score:", f1)
print("Validation Support:", support)

# # Precision-Recall Curve for validation set
# precision_curve, recall_curve, thresholds = precision_recall_curve(y_val, xgb_model.predict_proba(X_val)[:, 1])
# plt.plot(recall_curve, precision_curve)
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Precision-Recall Curve (Validation)')
# plt.show()

# Evaluate the model on the test set
y_pred_test = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy}")

precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred_test, average=None)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1-score:", f1)
print("Test Support:", support)

# Confusion Matrix for test set
cm = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix:\n", cm)

# # Precision-Recall Curve for test set
# precision_curve, recall_curve, thresholds = precision_recall_curve(y_test, xgb_model.predict_proba(X_test)[:, 1])
# plt.plot(recall_curve, precision_curve)
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.title('Precision-Recall Curve (Test)')
# plt.show()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from xgboost import XGBClassifier

# Initialize the XGBoost model
xgb_model = XGBClassifier(colsample_bytree = 1, n_estimators=200, learning_rate=0.1, max_depth=11, random_state=42)

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')

# Display the evaluation metrics
print("XGBoost Model Performance:")
print(f"F1 Score: {f1_xgb}")
print(f"Precision: {precision_xgb}")
print(f"Recall: {recall_xgb}")